<hr style="border: 2px solid coral"></hr>

# Practice : Matrix-free Jacobi method

<hr style="border: 2px solid coral"></hr>



### Example : Jacobi

Use the matrix-free Jacobi method to solve the 1d elliptic problem 

\begin{equation}
u''(x) = 1, \quad x \in [0,1]
\end{equation}

subject to boundary conditions $u(0) = u(1) = 0$.   

* Discretize the equation using the centered finite difference scheme at nodes. 

* Solve the problem on a mesh of size $N=8$, where $N$ is the number of mesh cells.  

* To check your answer, use the true solution $u(x) = x(x-1)/2$. 

Below is the Python solution.  For this notebook, write the solution in C. 

In [1]:
from numpy import *
from matplotlib.pyplot import *

from numpy.linalg import solve, norm

In [2]:
# Construct (N-1)x(N-1) linear system. 

def get_rhs(a,b,N):
    h = (b-a)/N
    F = h**2*ones((N-1,1))
    
    return F

def true_solution(N):
    x = linspace(0,1,N+1).reshape((N+1,1))[1:-1]
    return x*(x-1)/2

In [3]:
N = 8

a = 0
b = 1
F = get_rhs(a,b,N)

with printoptions(formatter={'float' : "{:12.8f}".format}):
    print("F = h^2 [1,1,1,...,1]")
    print(F)
    print("")

# True solution : Need to form A explicitly. 
u_true = true_solution(N)

with printoptions(formatter={'float' : "{:12.8f}".format}):
    print("True solution u = [u(x_1),u(x_2), ..., u(x_Nm1)] : ")
    print(u_true)
    print("")
    


F = h^2 [1,1,1,...,1]
[[  0.01562500]
 [  0.01562500]
 [  0.01562500]
 [  0.01562500]
 [  0.01562500]
 [  0.01562500]
 [  0.01562500]]

True solution u = [u(x_1),u(x_2), ..., u(x_Nm1)] : 
[[ -0.05468750]
 [ -0.09375000]
 [ -0.11718750]
 [ -0.12500000]
 [ -0.11718750]
 [ -0.09375000]
 [ -0.05468750]]



### Matrix-free Jacobi

In this approach, we formulate our Jacobi iteration as

\begin{equation}
\mathbf u_{k+1} = \mathbf u_k + D^{-1}(\mathbf F - A \mathbf u)
\end{equation}

We take advantage of the sparsity of $A$ to form the matrix-vector multiply to form the entry of $L = A\mathbf x$ row-by-row.  The loop is

    for i in range(1,N-2):
            L[i] = (x[i-1] -2*x[i] + x[i+1])
            
Entries $j = 0$ and $j=N-1$  are handled separately.

In [4]:
def apply_Laplacian(x):
    L = zeros(x.shape)
    m = x.shape[0]
    
    L[0] = (-2*x[0] + x[1])
    for i in range(1,m-1):
        L[i] = (x[i-1] -2*x[i] + x[i+1])
            
    L[-1] = (x[-2] - 2*x[-1])
    return L

# Version with A*x replaced by a loop
def jacobi_ver2(matvec,F,tol=1e-12,kmax=10000,prt=False):

    m = F.shape[0]
    xk = zeros(F.shape)
    itcount = kmax
    for k in range(kmax):
    
        # ---------------------
        # Update xkp1
        # ---------------------

        # Apply the Laplacian operator
        Ax = matvec(xk)

        dij = -2

        xkp1 = zeros(xk.shape)
        for i in range(m):
            ri = F[i] - Ax[i]
            xkp1[i] = xk[i] + ri/dij

        # ---------------------
        # Done with xkp1 update
        # ---------------------

        err = norm(xk-xkp1,inf)
        if prt:
            print("{:5d} {:12.4e}".format(k,err))

        if err < tol:
            xk = xkp1
            itcount = k
            break

        xk = xkp1
        
    return xk,itcount

In [5]:
# Use same parameters as in version 1

# Jacobi iteration : Version 2
u, itcount = jacobi_ver2(apply_Laplacian,F)

# Compute error
print("Iteration count (Jacobi, ver. 2) {:d}".format(itcount))
print("")


# print the error
err = abs(u_true - u)
if N < 32:
    with printoptions(formatter={'float' : "{:12.4e}".format}):
        print("Error u[i] - u_true[i]: ")
        print(err)
        print("")
        
# Compute the residual

Lu = apply_Laplacian(u)
res = abs(F - Lu)

with printoptions(formatter={'float' : "{:12.4e}".format}):
    print("Residual F - Lu")
    print(res)
        

Iteration count (Jacobi, ver. 2) 292

Error u[i] - u_true[i]: 
[[  4.1498e-12]
 [  7.6918e-12]
 [  1.0018e-11]
 [  1.0878e-11]
 [  1.0018e-11]
 [  7.6918e-12]
 [  4.1498e-12]]

Residual F - Lu
[[  6.0774e-13]
 [  1.2154e-12]
 [  1.4672e-12]
 [  1.7189e-12]
 [  1.4671e-12]
 [  1.2155e-12]
 [  6.0771e-13]]


<hr style="border:2px solid black"></hr>

## Example : Matrix-free Jacobi (serial version)

<hr style="border:2px solid black"></hr>

Fill in TODOs to write a serial Jacobi method. 

### Comments 

* Pay attention to where the $h^2$ goes.  We have multiplied the RHS by $h^2$, so that it is not used in the "matrix-vector" multiply. 



In [1]:
%%file jacobi_01.c

#include <stdio.h>
#include <stdlib.h>

#include <math.h>

double* allocate_1d(int n, int m)
{
    double *mem = (double*) malloc((n + 2*m)*sizeof(double));
    return &mem[m];
}

void free_1d(double **x, int m)
{
    free(&(*x)[-m]);
    *x = NULL;
}


void get_rhs(int N, double a, double b, double *F)
{
    double h = (b-a)/N;
    for(int i = 0; i < N+1; i++)
    {
        F[i] = h*h;        
    }    
}

double utrue(double x)
{
    double utrue = x*(x-1)/2;
    
    return utrue;
}

void apply_laplacian(int N, double *u, double *L)
{
    double g0 = 0;
    u[-1] = 2*g0 -u[1];
    
    double g1 = 0;
    u[N+1] = 2*g1 - u[N-1];
    
    for(int i = 0; i<N+1; i++)
        L[i] = (u[i-1] - 2*u[i] + u[i+1]);
}

void jacobi_ver2(int N, double *F, double *u, double tol, int kmax, int prt)
{
    for (int j = 0; j<N+1; j++)
        u[j] = 0;
    /* Jacobi iteration */
    double *Lu = allocate_1d(N+1,0);
    double *uk = allocate_1d(N+1,0);
    double *ukp1 = allocate_1d(N+1,1);
    for (int k = 0; k<kmax; k++)
    {

        apply_laplacian(N, a, b, u, Lu);
        double di = -2;
#if 0
        
#endif
    }
}

int main(int argc, char** argv)
{
    
    double a = 0; 
    double b = 1;
    
    int N = 8;
    double tol = 1e-12;
    int kmax = 1000;
    int prt = 1;
        
    double *u = allocate_1d(N,1);
    double *F = allocate_1d(N,0);
    
    get_rhs(N,a,b,F);
    
    jacobi_ver2(N,F,u,tol,kmax,prt);
        
    // # TODO : Compute the error
    
    // # TODO : Compute the residual
    
    
    return 0;
}    

Writing jacobi_01.c


In [1]:
%%bash

rm -rf jacobi_01.o jacobi_01

gcc -o jacobi_01 jacobi_01.c

./jacobi_01

jacobi_01.c:59:28: error: use of undeclared identifier 'a'
        apply_laplacian(N, a, b, u, Lu);
                           ^
jacobi_01.c:59:31: error: use of undeclared identifier 'b'
        apply_laplacian(N, a, b, u, Lu);
                              ^
2 errors generated.
bash: line 6: ./jacobi_01: No such file or directory


CalledProcessError: Command 'b'\nrm -rf jacobi_01.o jacobi_01\n\ngcc -o jacobi_01 jacobi_01.c\n\n./jacobi_01\n'' returned non-zero exit status 127.